# Internet checksum

Este notebook explica cómo se calcula el checksum que utilizan las cabeceras de IPv4, ICMPv4, IGMP, UDP o TCP. Se le conoce habitualmente como «checksum de Internet» o *Internet checksum*.

El checksum permite detectar cambios (incluso de un solo bit) en la transmisión del mensaje. Consiste esencialmente en interpretar el mensaje como una secuencia de enteros de 16 bits con ajustes para que el resultado también sea un entero de 16 bits.

## Un ejemplo

Como ejemplo, vamos a tomar la cabecera de un paquete IP que encapsula un mensaje ICMP. Inicialmente el campo checksum debe contener un 0.

In [6]:
ip_header = [
    0x45, 0x00,             # Versión, IHL, Tipo de Servicio
    0x00, 0x54,             # Longitud Total (84 bytes)
    0x00, 0x00,             # Identificación
    0x40, 0x00,             # Flags, Offset de Fragmento
    0x40, 0x01,             # TTL, Protocolo (ICMP)
    0x00, 0x00,             # CHECKSUM (0x0000 por ahora)
    0xC0, 0xA8, 0x00, 0x01, # IP origen (192.168.0.1)
    0xC0, 0xA8, 0x00, 0xC7, # IP destino (192.168.0.199)
]

Se divide el mensaje (la cabecera IP en este caso) en palabras de 16 bits y se suman haciendo plegado (folding) del acarreo. Es decir, si la suma excede `0xFFFF` el desbordamiento se suma al resultado. Por ejemplo, al sumar 0xFFFE + 4 = 0x10002, se "pliega" como 0x0002 + 1 = 0x0003.

Aplicado a la cabecera anterior:

In [7]:
words = [
    0x4500,
    0x0054,
    0x0000,
    0x4000,
    0x4001,
    0x0000,
    0xC0A8,
    0x0001,
    0xC0A8,
    0x00C7
]

| Op 1      |   Op 2      | Suma      |
|-----------|------------:|----------:|
| 0x4500    | + 0x0054    |  0x4554   |
| 0x4554    | + 0x0000    |  0x4554   |
| 0x4554    | + 0x4000    |  0x8554   |
| 0x8554    | + 0x4001    |  0xC555   |
| 0xC555    | + 0x0000    |  0xC555   |
| 0xC555    | + 0xC0A8    | 0x185FD   |
| folding   | +   0x01    |  0x85FE   |
| 0x85FE    | + 0x0001    |  0x85FF   |
| 0x85FF    | + 0xC0A8    | 0x146A7   |
| folding   | +   0x01    |  0x46A8   |
| 0x46A8    | + 0x00C7    |  0x476f   |

Finalmente se calcula el complemento a 1 y tenemos el valor del checksum:

In [8]:
hex(~0x476f & 0xFFFF)

'0xb890'

Este valor es big endian, es decir, es el orden con el que debe ser colocado en el campo checksum del mensaje. Al recibir el mensaje para verificar el checksum, se repite el calculo (que incluye el checksum calculado) y el resultado debe ser `0` si el mensaje no ha sufrido cambios.

## El algoritmo

La RFC 1071 proporciona una implementación en C del algoritmo que podemos usar:

```c
/* Compute Internet Checksum for "count" bytes
*         beginning at location "addr".
*/
register long sum = 0;

while( count > 1 )  {
    /*  This is the inner loop */
        sum += * (unsigned short) addr++;
        count -= 2;
}

/*  Add left-over byte, if any */
if( count > 0)
        sum += * (unsigned char *) addr;

/*  Fold 32-bit sum to 16 bits */
while (sum>>16)
    sum = (sum & 0xffff) + (sum >> 16);

checksum = ~sum;
```

Analizando este código puedes ver que implica varios pasos, que corresponden al algoritmo indicado, aunque con alguna diferencia (obviamente compatible).

- Suma palabras de 16 bits tomando 2 bytes cada vez gracias al molde de short.
- Si la cantidad de datos (count) es impar suma el último byte.
- En lugar de hacer folding en cada suma, lo hace al acabar. Y lo hace con un bucle hasta que no quede exceso por encima de los 16 bits
- Por último calcula el complemento a 1

Pero cuidado, ese programa no es portable! Es para una arquitectura little endian y requiere intercambiar los bytes al terminar (p.ej. con `htons`). Lo sabemos porque para una secuencia de tamaño impar suma ese último byte como LSB.

Teniendo todo eso en cuenta, podemos proponer una función equivalente en Python, si bien esta versión es portable y retorna el resultado como big endian en cualquier caso:

In [9]:
def inet_checksum(data: bytes) -> int:
    total = 0

    for i in range(0, len(data) - 1, 2):
        total += (data[i] << 8) + data[i+1]

    if len(data) % 2 == 1:
        total += data[-1] << 8

    while (total >> 16):
        total = (total & 0xffff) + (total >> 16)

    return ~total & 0xffff

Aplicada a la cabecera del principio:

In [10]:
hex(inet_checksum(bytes(ip_header)))

'0xb890'